In [1]:
# https://github.com/Kaggle/kaggle-environments/blob/master/kaggle_environments/envs/hungry_geese/hungry_geese.py

In [2]:
from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, \
                                                                row_col, adjacent_positions, translate, min_distance

from kaggle_environments import make
from random import choice
import numpy as np
from copy import deepcopy
import pickle

Loading environment football failed: No module named 'gfootball'


In [3]:
import numpy as np
import random
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

from collections import deque
import sys

In [4]:
np.set_printoptions(linewidth=100)

In [5]:
row_col(40, 11)

(3, 7)

In [6]:
class GreedyAgent:
    def __init__(self):
        
        self.last_action = None
        self.observations = []

    def __call__(self, observation: Observation, configuration: Configuration):
        self.configuration = configuration
        
        board = np.zeros(self.configuration.rows*self.configuration.columns)
        board_shape = (self.configuration.rows, self.configuration.columns)
        
        board_heads = deepcopy(board)
        board_bodies = deepcopy(board)
        board_rewards = deepcopy(board)
        
        
        rows, columns = self.configuration.rows, self.configuration.columns

        food = observation.food
        geese = observation.geese
        
        
        opponents = [
            goose
            for index, goose in enumerate(geese)
            if index != observation.index and len(goose) > 0
        ]

        
        opponent_heads = [opponent[0] for opponent in opponents]
        # Don't move adjacent to any heads
        head_adjacent_positions = {
            opponent_head_adjacent
            for opponent_head in opponent_heads
            for opponent_head_adjacent in adjacent_positions(opponent_head, columns, rows)
        }
        
        tail_adjacent_positions ={
            opponent_tail_adjacent
            for opponent in opponents
            for opponent_tail in [opponent[-1]]
            for opponent_tail_adjacent in adjacent_positions(opponent_tail, columns, rows)
        }
        # Don't move into any bodies
        #bodies, heads = [position for goose in geese for position in goose]
        
        heads = [i[0] for i in geese if len(i)>1]
        bodies = [item for sublist in geese for item in sublist]
        
        board_bodies[list(bodies)] = 1
        board_heads[heads] = 1

        # Move to the closest food
        position = geese[observation.index][0]
        actions = {
            action: min_distance(new_position, food, columns)
            for action in Action
            for new_position in [translate(position, action, columns, rows)]
            if (
                new_position not in head_adjacent_positions and
                new_position not in bodies and
                (self.last_action is None or action != self.last_action.opposite())
            )
        }

        action = min(actions, key=actions.get) if any(actions) else choice([action for action in Action])
        
        
        cur_obs = {}
        cur_obs['head_adjacent_positions'] = head_adjacent_positions
        cur_obs['bodies'] = bodies
        cur_obs['board_bodies'] = board_bodies.reshape(board_shape)
        cur_obs['board_heads'] = board_heads.reshape(board_shape)
        cur_obs['tails'] = tail_adjacent_positions
        cur_obs['actions'] = actions
        cur_obs['action'] = action
        cur_obs['last_action'] = self.last_action
#         cur_obs['goose_size'] = player_goose_len
#         cur_obs['board'] = board
        cur_obs['cur_action'] = action
        self.observations.append(cur_obs)
        
        self.last_action = action
        return action.name


cached_greedy_agents = {}


def greedy_agent(obs, config):
    index = obs["index"]
    if index not in cached_greedy_agents:
        cached_greedy_agents[index] = GreedyAgent(Configuration(config))
    return cached_greedy_agents[index](Observation(obs))

In [7]:
def geese_heads(obs_dict, config_dict):
    """
    Return the position of the geese's heads
    """
    configuration = Configuration(config_dict)

    observation = Observation(obs_dict)
    player_index = observation.index
    player_goose = observation.geese[player_index]
    player_head = player_goose[0]
    player_row, player_column = row_col(player_head, configuration.columns)
    positions = []
    for geese in observation.geese:
        if len(geese)>0:
            geese_head = geese[0]
            row, column = row_col(geese_head, configuration.columns)
        else:
            row = None
            column = None
        positions.append((row, column))
    return positions

def get_last_actions(previous_geese_heads, heads_positions):

    def get_last_action(prev, cur):
        last_action = None

        prev_row = prev[0]
        prev_col = prev[1]
        cur_row = cur[0]
        cur_col = cur[1]

        if cur_row is not None:
            if (cur_row-prev_row == 1) | ((cur_row==0) & (prev_row==6)):
                last_action = Action.SOUTH.name
            elif (cur_row-prev_row == -1) | ((cur_row==6) & (prev_row==0)):
                last_action = Action.NORTH.name
            elif (cur_col-prev_col == 1) | ((cur_col==0) & (prev_col==10)):
                last_action = Action.EAST.name
            elif (cur_col-prev_col == -1) | ((cur_col==10) & (prev_col==0)):
                last_action = Action.WEST.name

        return last_action

    if len(previous_geese_heads) == 0:
        actions = [Action.SOUTH.name, Action.NORTH.name, Action.EAST.name, Action.WEST.name]
        nb_geeses = len(heads_positions)
        last_actions = [actions[np.random.randint(4)] for _ in range(nb_geeses)]
    else:   
        last_actions = [get_last_action(*pos) for pos in zip(previous_geese_heads, heads_positions)]

    return last_actions
    
def central_state_space(obs_dict, config_dict, prev_head):
    """
    Recreating a board where my agent's head in the middle of the board 
    (position (4,5)), and creating features accordingly
    """
    
    configuration = Configuration(config_dict)

    observation = Observation(obs_dict)
    player_index = observation.index
    player_goose = observation.geese[player_index]
    if len(player_goose)==0:
        player_head = prev_head
    else:
        player_head = player_goose[0]
    player_row, player_column = row_col(player_head, configuration.columns)
    row_offset = player_row - 3
    column_offset = player_row - 5

    foods = observation['food']

    def centralize(row, col):
        if col > player_column:
            new_col = (5 + col - player_column) % 11
        else:
            new_col = 5 - (player_column - col)
            if new_col < 0:
                new_col += 11

        if row > player_row:
            new_row = (3 + row - player_row) % 7
        else:
            new_row = 3 - (player_row - row)
            if new_row < 0:
                new_row += 7
        return new_row, new_col

    food1_row, food1_column = centralize(*row_col(foods[0], configuration.columns))
    food2_row, food2_column = centralize(*row_col(foods[1], configuration.columns))

    food1_row_feat = float(food1_row - 3)/3 if food1_row>=3 else float(food1_row - 3)/3
    food2_row_feat = float(food2_row - 3)/3 if food2_row>=3 else float(food2_row - 3)/3

    food1_col_feat = float(food1_column - 5)/5 if food1_column>=5 else float(food1_column - 5)/5
    food2_col_feat = float(food2_column - 5)/5 if food2_column>=5 else float(food2_column - 5)/5
    food_feats = np.array([food1_row_feat, food2_row_feat, food1_col_feat, food2_col_feat])

    # Create the grid
    board = np.zeros([7, 11])
    # Add food to board
#     board[food1_row, food1_column] = 1
#     board[food2_row, food2_column] = 1

    for ind, goose in enumerate(observation.geese):
        if ind!= player_index:
            if len(goose)>0:

                ap = adjacent_positions(goose[0], 11, 7)
                for p in ap:
                    row, col = centralize(*row_col(p, configuration.columns))    
                    board[row,col]=-.33             

        for pos in goose:
            if len(goose)>0:
                # set bodies to 1
                row, col = centralize(*row_col(pos, configuration.columns))
                board[row, col]=-1
                
   
        if len(goose)>1:
            # Set tails to -0.1 if the goose has a tail
            row, col = centralize(*row_col(goose[-1], configuration.columns))
            board[row, col]=-0.1    
                
    board[3, 5] = 0
    return board, food_feats

In [8]:
adjacent_positions(2, 11, 7)

[68, 3, 13, 1]

In [9]:
env = make("hungry_geese", debug=True)
config = env.configuration

state_dict = env.reset(num_agents=4)[0]
observation = state_dict['observation']
action = state_dict['action']

prev_head = 0

In [10]:
observation

{'remainingOverageTime': 60,
 'step': 0,
 'geese': [[62], [9], [61], [18]],
 'food': [3, 5],
 'index': 0}

In [11]:
central_state_space(observation, config, prev_head)

(array([[ 0.  ,  0.  ,  0.  ,  0.  ,  0.  , -0.33,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  0.  , -0.33,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  , -0.33, -1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  0.  , -0.33,  0.  ,  0.  , -0.33,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  , -0.33, -0.33, -1.  , -0.33,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  0.  , -0.33, -1.  , -0.33, -0.33,  0.  ,  0.  ,  0.  ]]),
 array([ 0.66666667,  0.66666667, -0.8       , -0.4       ]))

In [12]:
env.render(mode="ipython")

In [13]:
def min_dir(p1, p2, max_p):
    """
    min distance and direction from p1
    """
    direction = 'left' # left by default
    d1 = abs(p1 - p2) # Distance going across board
    d2 = min(abs(p1-0), abs(p1 - max_p)) + min(abs(p2-0), abs(p2 - max_p)) # Distance wrapping around board
    
    if p1>p2 and d1<d2:
        direction = 'left'
        
    elif p1>p2 and d1>d2:
        direction ='right'
    
    elif p2>p1 and d1<d2:
        direction = 'right'
    
    elif p2>p1 and d1>d2:
        direction = 'left'
    
    dir_vec = np.zeros(2)
        
    if direction=='left':
        dir_vec[0]=1
    
    else:
        dir_vec[1]=1
    
    min_dist = np.array([min(d1, d2)])/max_p
    return dir_vec, min_dist
        

### Test min dir

In [14]:
cols = 11
p1 = 2
p2 = 5

min_dir(p1,p2,cols)

(array([0., 1.]), array([0.27272727]))

In [15]:
cols = 11
p1 = 2
p2 = 9

min_dir(p1,p2,cols)

(array([1., 0.]), array([0.36363636]))

In [16]:
cols = 11
p1 = 9
p2 = 2

min_dir(p1, p2, cols)

(array([0., 1.]), array([0.36363636]))

In [17]:
class StateTranslator_Central:
    """
    Returns a board where we are always at the center
    """
    
    def __init__(self):
        
        self.last_action = None
        self.step_count = 0
        self.last_goose_length = 1
        self.last_goose_ind = 0
        self.observations = []
        
    def set_last_action(self, last_action):
        self.last_action = last_action
        
        
    def __get_last_action_vec(self):
        action_vec = np.zeros(4)
        
        if self.last_action == 'NORTH':
            action_vec[0] = 1
        elif self.last_action == 'SOUTH':
            action_vec[1] = 1
        elif self.last_action == 'EAST':
            action_vec[2] = 1
        elif self.last_action == 'WEST':
            action_vec[3] = 1
        
        return action_vec
    
    def translate_action_to_text(self, action):
        
        h = {0 : 'WEST',
             1: 'EAST',
             2: 'NORTH',
             3: 'SOUTH'}
        
        return h[action]

    def translate_text_to_int(self, action):
        h = {'WEST':0,
            'EAST':1,
            'NORTH':2,
            'SOUTH':3}
        
        return h[action]
    
    
    def update_length(self):
        self.last_goose_length = self.current_goose_length
        
    
    def get_state(self, observation, config):
        
        #### This is exception handling for if our goose died this turn, to use the last
        ### known index as its postion for the state centralizer
        geese = observation['geese']
        if len(geese[observation['index']])>0:
            self.last_goose_ind = geese[observation['index']][0]
            self.my_goose = geese[observation['index']][0]
            
        
        board, food_feats = central_state_space(observation, config, self.last_goose_ind)
        self.step_count = observation['step']
        
        
        self.current_goose_length = len(geese[observation['index']])  
        
        
        food = observation['food']
        dir_vec1x, min_dist1x = min_dir(self.my_goose, food[0] , 11)
        dir_vec1y, min_dist1y = min_dir(self.my_goose, food[0] , 7)
        
        dir_vec2x, min_dist2x = min_dir(self.my_goose, food[1] , 11)   
        dir_vec2y, min_dist2y = min_dir(self.my_goose, food[1] , 7)  
        food_vec = np.concatenate((dir_vec1x, dir_vec1y, dir_vec2x, dir_vec2y,
                            min_dist1x, min_dist1y, min_dist2x, min_dist2y))
        ####
        biggest_goose = 0
        alive_geese = 0
        for ind, goose in enumerate(geese):
            if len(goose)>biggest_goose:
                biggest_goose = len(goose)
            if ind != observation['index'] and len(goose)>0:
                alive_geese+=1
        
        state = np.array([])
        state = np.append(state, self.__get_last_action_vec())
        state = np.append(state, self.current_goose_length/15)
        state = np.append(state, biggest_goose/15)
        state = np.append(state, alive_geese/4)
        state = np.append(state, self.step_count/200)
        state = np.append(state, food_vec)
        state = np.append(state, board.flatten())
        
        state = state.flatten()
        
        return state
    
    def calculate_reward(self, observation):
        
        current_geese = observation['geese']
        prev = self.last_goose_length
        cur = len(current_geese[observation['index']])  
        
        reward = -1       
    
        ### If we grow, reward is 100
        if cur>prev:
            reward = 100
        
        else:
            reward = -1
        
        ### If we die -150
        if cur == 0:
            reward = -150
            
        ### see if any geese are alive

        alive_geese = 0
        for ind, goose in enumerate(current_geese):
            if ind != observation['index'] and len(goose)>0:
                alive_geese+=1
        
        # If we are the last one standing
        if alive_geese == 0 and cur>0:
            reward = 500
            
        ### if the game ends and we are the biggest
        if self.step_count == 200:
            biggest_goose = 0
            biggest_goose_ind = None
            for ind, goose in enumerate(current_geese):
                if len(goose)>biggest_goose:
                    biggest_goose = len(goose)
                    biggest_goose_ind = ind
            
            if biggest_goose_ind == observation['index']:
                reward = 500
        
        return reward
        

In [18]:
st_test = StateTranslator_Central()

In [19]:
env = make("hungry_geese", debug=True)
config = env.configuration

state_dict = env.reset(num_agents=4)[0]
observation = state_dict['observation']
action = state_dict['action']

In [20]:
state_dict

{'action': 'NORTH',
 'reward': 0,
 'info': {},
 'observation': {'remainingOverageTime': 60,
  'step': 0,
  'geese': [[39], [4], [2], [34]],
  'food': [22, 42],
  'index': 0},
 'status': 'ACTIVE'}

In [21]:
st_test.set_last_action("SOUTH")
state_test = st_test.get_state(observation, config)
state_test

array([ 0.        ,  1.        ,  0.        ,  0.        ,  0.06666667,  0.06666667,  0.75      ,
        0.        ,  1.        ,  0.        ,  1.        ,  0.        ,  0.        ,  1.        ,
        0.        ,  1.        ,  1.54545455,  2.42857143,  0.27272727,  0.42857143, -0.33      ,
       -1.        , -0.33      , -1.        , -0.33      ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.33      ,  0.        , -0.33      ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
       -0.33      ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -1.        , -0.33      ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
       -0.33      , -0.33      ,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        , 

In [22]:
state_test.shape

(97,)

In [23]:
st_test.set_last_action = 'NORTH'
st_test.translate_text_to_int('EAST')

1

In [24]:
st_test.calculate_reward(observation)

-1

In [64]:
class dqnAgent:
    """
    Given an environment state, choose an action, and learn from the reward
    https://towardsdatascience.com/reinforcement-learning-w-keras-openai-dqns-1eed3a5338c
    https://towardsdatascience.com/deep-q-learning-tutorial-mindqn-2a4c855abffc
    https://www.researchgate.net/post/What-are-possible-reasons-why-Q-loss-is-not-converging-in-Deep-Q-Learning-algorithm
    """

    def __init__(self, model=None, epsilon = 1.0, epsilon_min = 0.15):

        self.StateTrans = StateTranslator_Central()
        self.state_shape  = 97
        print('my state shape is:', self.state_shape)
        self.memory  = deque(maxlen=5000)
        self.gamma = 0.95
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = 0.999
        self.learning_rate = 0.001
        self.tau = .125

        if model == None:
            self.model = self.create_model()
        else:
            self.model = model
        self.target_model = self.create_model()

    def create_model(self):
        model   = Sequential()
        model.add(Dense(2000, input_dim=self.state_shape, activation="relu"))
        model.add(Dense(1000, activation="relu"))
        model.add(Dense(500, activation="relu"))
        model.add(Dense(1000, activation="relu"))
        model.add(Dense(500, activation="relu"))
        model.add(Dense(100, activation="relu"))
        model.add(Dense(4))
        model.compile(loss="MSE",
            optimizer=Adam(lr=self.learning_rate))
        return model

    def act(self, state):
        self.epsilon *= self.epsilon_decay
        self.epsilon = max(self.epsilon_min, self.epsilon)
        if np.random.random() < self.epsilon:
            # Set random choice to north or east as the agent is not moving in these directions
            return random.choice([0,1,2,3])

        action_values = self.model.predict(state.reshape(-1, self.state_shape))[0]
        action = np.argmax(action_values)

        return action
    
    def translate_state(self, observation, configuration):
        state = self.StateTrans.get_state(observation, configuration)
        return state

    def __call__(self, observation, configuration):
        
        state = self.translate_state(observation, configuration)
        action = self.act(state)
        # State translator will take in 0, 1, 2, 3 and return straight, left or right, which in turn will 
        # be translated into a kaggle Action
        action_text = self.StateTrans.translate_action_to_text(action)
        
        # Update our step number and actions
        self.StateTrans.set_last_action(action_text)
        
        
        return action_text
    
    def remember(self, state, action, reward, new_state, done):
        self.memory.append([state, action, reward, new_state, done])

    def replay(self):
        batch_size = 64
        if len(self.memory) < batch_size:
            return

        samples = random.sample(self.memory, batch_size)
        ########################
        # This can be sped up significantly, but processing all samples in batch rather than 1 at a time
        ####################
        states = np.array([])
        actions = np.array([])
        rewards = []
        dones = []
        new_states = np.array([])
        targets = np.array([])
        
        for sample in samples:
            state, action, reward, new_state, done = sample
            
            states = np.append(states, state)
            actions = np.append(actions, action)
            rewards.append(reward)
            new_states = np.append(new_states, new_state)
            dones.append(done)
        
        new_states = new_states.reshape(batch_size, self.state_shape)
        targets = self.target_model.predict(states.reshape(batch_size, self.state_shape))
        targets =  targets.reshape(batch_size, 4)
        for i in range(batch_size):    
            if dones[i]:
                targets[i][int(actions[i])] = rewards[i]

            else:
                Q_future = max(self.target_model.predict(new_states[i].reshape(-1, self.state_shape))[0])
#                 print('targets i', targets[i])
#                 print('actions[i]', actions[i])
                targets[i][int(actions[i])] = rewards[i] + Q_future * self.gamma
               
        
        self.model.fit(states.reshape(batch_size, self.state_shape), targets,
                       epochs=1, verbose=0)

    def target_train(self):
        weights = self.model.get_weights()
        target_weights = self.target_model.get_weights()
        for i in range(len(target_weights)):
            target_weights[i] = weights[i] * self.tau + target_weights[i] * (1 - self.tau)
        self.target_model.set_weights(target_weights)

    def save_model(self, fn):
        self.model.save(fn)


In [65]:
class ActorCritic:
    
    def __init__(self):
        pass

### Training script

In [ ]:
steps_per_ep = 200
num_episodes = 10000

env = make("hungry_geese", debug=True)
config = env.configuration
train_name = 'central_agent_3'
mod_num = 0 # Which trial to load 

#model = keras.models.load_model(f'central_agent_2/trial-{mod_num}')

dqn = dqnAgent(#model=model,
              #epsilon = 0.10,
              #epsilon_min = 0.10
                )

agent2 = GreedyAgent()
agent3 = GreedyAgent()
agent4 = GreedyAgent()

agents = [dqn, agent2, agent3, agent4]

results_dic = {}
for ep in range(num_episodes):
    print('episode number: ', ep+mod_num)
    state_dict = env.reset(num_agents=4)[0]
    observation = state_dict['observation']
    my_goose_ind = observation['index']
    
    dqn.StateTrans.set_last_action(None)
    dqn.StateTrans.step_count = 0
    dqn.StateTrans.last_goose_length = 1
    cur_state = dqn.StateTrans.get_state(observation, config)
    
    
    done = False
    for step in range(steps_per_ep):
        actions = []
        for ind, agent in enumerate(agents):
            obs_copy = deepcopy(observation)
            obs_copy['index'] = ind
            action = agent(obs_copy, config)
            actions.append(action)
        
        state_dict = env.step(actions)[0]
        observation = state_dict['observation']
        print(observation)
        
        action = state_dict['action']
        status = state_dict['status']
        
        
        action_for_model = dqn.StateTrans.translate_text_to_int(action)
        new_state = dqn.StateTrans.get_state(observation, config)
        
        # Set rewards based on if value was gained or lost
        reward = dqn.StateTrans.calculate_reward(observation)
        # Update our goose length based on prev state
        dqn.StateTrans.update_length()
       

        if status != "ACTIVE":
            done = True
        
        
        #Temp for just training agent to go get food
        
#         if reward<-1:
#             reward = -1
        print('reward: ', reward)
        dqn.remember(cur_state, action_for_model, reward, new_state, done)
        
        cur_state = new_state
            
        # Check if my goose died

            
        if done:
            print('Done, Step: ', step)
            print('status, ', status)
            results_dic[ep] = reward
            
            if ep % 50 == 0:
                directory = train_name
                dqn.save_model(directory + f"/trial-{ep+mod_num}")
                with open(directory + "/results_dic.pkl", 'wb') as f:
                    pickle.dump(results_dic, f)
            break
        

        if step%5 == 0:
            dqn.replay()        
            dqn.target_train()

#         if step%50 == 0:
#             print(f'We survived {step} steps')
#             directory = train_name
#             dqn.save_model(directory + f"/trial-{ep}")
        


my state shape is: 97
episode number:  0
{'remainingOverageTime': 60, 'step': 1, 'geese': [[19], [44], [28], [8]], 'food': [13, 68], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 2, 'geese': [[20], [54], [27], [7]], 'food': [13, 68], 'index': 0}
reward:  -1
Opposite action: (0, <Action.WEST: 4>, <Action.EAST: 2>)
{'remainingOverageTime': 60, 'step': 3, 'geese': [[], [53], [26], [6]], 'food': [13, 68], 'index': 0}
reward:  -150
Done, Step:  2
status,  DONE
INFO:tensorflow:Assets written to: central_agent_3/trial-0\assets
episode number:  1
{'remainingOverageTime': 60, 'step': 1, 'geese': [[43], [50], [76], [54]], 'food': [62, 31], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 2, 'geese': [[42], [39], [65], [43]], 'food': [62, 31], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 3, 'geese': [[53], [28], [54], [32]], 'food': [62, 31], 'index': 0}
reward:  -1
Opposite action: (0, <Action.NORTH: 1>, <Action.SOUTH: 3>)
{'remainingOverageTime': 60, '

Opposite action: (0, <Action.EAST: 2>, <Action.WEST: 4>)
{'remainingOverageTime': 60, 'step': 2, 'geese': [[], [60], [40], [32]], 'food': [23, 72], 'index': 0}
reward:  -150
Done, Step:  1
status,  DONE
episode number:  11
{'remainingOverageTime': 60, 'step': 1, 'geese': [[16], [72], [57], [26]], 'food': [40, 59], 'index': 0}
reward:  -1
Opposite action: (0, <Action.EAST: 2>, <Action.WEST: 4>)
{'remainingOverageTime': 60, 'step': 2, 'geese': [[], [73], [58], [27]], 'food': [40, 59], 'index': 0}
reward:  -150
Done, Step:  1
status,  DONE
episode number:  12
{'remainingOverageTime': 60, 'step': 1, 'geese': [[34], [17], [12], [70]], 'food': [40, 68], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 2, 'geese': [[35], [18], [13], [71]], 'food': [40, 68], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 3, 'geese': [[46], [29], [24], [5]], 'food': [40, 68], 'index': 0}
reward:  -1
Opposite action: (0, <Action.NORTH: 1>, <Action.SOUTH: 3>)
{'remainingOverageTime': 60,

{'remainingOverageTime': 60, 'step': 2, 'geese': [[69], [62], [68], [26]], 'food': [32, 16], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 3, 'geese': [[58], [73], [2], [37]], 'food': [32, 16], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 4, 'geese': [[57], [74], [3], [38]], 'food': [32, 16], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 5, 'geese': [[46], [63], [69], [27]], 'food': [32, 16], 'index': 0}
reward:  -1
Opposite action: (0, <Action.SOUTH: 3>, <Action.NORTH: 1>)
{'remainingOverageTime': 60, 'step': 6, 'geese': [[], [52], [58], [16, 27]], 'food': [32, 43], 'index': 0}
reward:  -150
Done, Step:  5
status,  DONE
episode number:  26
{'remainingOverageTime': 60, 'step': 1, 'geese': [[37], [43], [64], [9]], 'food': [56, 38], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 2, 'geese': [[26], [33], [65], [10]], 'food': [56, 38], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 3, 'geese': [[15], [34], [55], [0]]

{'remainingOverageTime': 60, 'step': 1, 'geese': [[72], [7], [24], [6]], 'food': [31, 23], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 2, 'geese': [[61], [73], [13], [72]], 'food': [31, 23], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 3, 'geese': [[50], [62], [2], [61]], 'food': [31, 23], 'index': 0}
reward:  -1
Opposite action: (0, <Action.SOUTH: 3>, <Action.NORTH: 1>)
{'remainingOverageTime': 60, 'step': 4, 'geese': [[], [51], [68], [50]], 'food': [31, 23], 'index': 0}
reward:  -150
Done, Step:  3
status,  DONE
episode number:  40
{'remainingOverageTime': 60, 'step': 1, 'geese': [[14], [62], [26], [57]], 'food': [31, 75], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 2, 'geese': [[3], [51], [15], [46]], 'food': [31, 75], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 3, 'geese': [[2], [40], [4], [35]], 'food': [31, 75], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 4, 'geese': [[1], [29], [70], [24]], 'food

{'remainingOverageTime': 60, 'step': 2, 'geese': [[0], [46], [21], [58]], 'food': [16, 45], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 3, 'geese': [[66], [35], [10], [47]], 'food': [16, 45], 'index': 0}
reward:  -1
Opposite action: (0, <Action.SOUTH: 3>, <Action.NORTH: 1>)
{'remainingOverageTime': 60, 'step': 4, 'geese': [[], [24], [76], [36]], 'food': [16, 45], 'index': 0}
reward:  -150
Done, Step:  3
status,  DONE
episode number:  53
{'remainingOverageTime': 60, 'step': 1, 'geese': [[43], [56], [1], [50]], 'food': [4, 27], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 2, 'geese': [[32], [45], [67], [39]], 'food': [4, 27], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 3, 'geese': [[22], [44], [66], [38]], 'food': [4, 27], 'index': 0}
reward:  -1
Goose Collision: SOUTH
Goose Collision: NORTH
{'remainingOverageTime': 60, 'step': 4, 'geese': [[], [], [55], [27, 38]], 'food': [4, 2], 'index': 0}
reward:  -150
Done, Step:  3
status,  DONE
epi

...

{'remainingOverageTime': 60, 'step': 32, 'geese': [[48, 47, 46, 45, 44, 33, 43, 42], [65], [], []], 'food': [49, 71], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 33, 'geese': [[49, 48, 47, 46, 45, 44, 33, 43, 42], [55], [], []], 'food': [71, 51], 'index': 0}
reward:  100
{'remainingOverageTime': 60, 'step': 34, 'geese': [[50, 49, 48, 47, 46, 45, 44, 33, 43], [56], [], []], 'food': [71, 51], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 35, 'geese': [[39, 50, 49, 48, 47, 46, 45, 44, 33], [57], [], []], 'food': [71, 51], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 36, 'geese': [[28, 39, 50, 49, 48, 47, 46, 45, 44], [58], [], []], 'food': [71, 51], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 37, 'geese': [[29, 28, 39, 50, 49, 48, 47, 46, 45], [59], [], []], 'food': [71, 51], 'index': 0}
reward:  -1
Opposite action: (0, <Action.WEST: 4>, <Action.EAST: 2>)
{'remainingOverageTime': 60, 'step': 38, 'geese': [[], [70], [], []], 

{'remainingOverageTime': 60, 'step': 2, 'geese': [[33, 43], [69], [39], [1]], 'food': [28, 62], 'index': 0}
reward:  100
Opposite action: (0, <Action.WEST: 4>, <Action.EAST: 2>)
{'remainingOverageTime': 60, 'step': 3, 'geese': [[], [58], [28, 39], [67]], 'food': [62, 12], 'index': 0}
reward:  -150
Done, Step:  2
status,  DONE
episode number:  1213
{'remainingOverageTime': 60, 'step': 1, 'geese': [[74], [39], [37], [31]], 'food': [5, 30], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 2, 'geese': [[63], [28], [26], [20]], 'food': [5, 30], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 3, 'geese': [[52], [17], [15], [9]], 'food': [5, 30], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 4, 'geese': [[41], [6], [4], [75]], 'food': [5, 30], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 5, 'geese': [[40], [72], [70], [64]], 'food': [5, 30], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 6, 'geese': [[29], [61], [59], [53]]

{'remainingOverageTime': 60, 'step': 7, 'geese': [[60, 59], [43, 54], [46], [69, 3]], 'food': [52, 39], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 8, 'geese': [[61, 60], [32, 43], [35], [58, 69]], 'food': [52, 39], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 9, 'geese': [[62, 61], [21, 32], [24], [47, 58]], 'food': [52, 39], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 10, 'geese': [[51, 62], [10, 21], [13], [36, 47]], 'food': [52, 39], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 11, 'geese': [[52, 51, 62], [0, 10], [14], [37, 36]], 'food': [39, 59], 'index': 0}
reward:  100
{'remainingOverageTime': 60, 'step': 12, 'geese': [[53, 52, 51], [66, 0], [3], [26, 37]], 'food': [39, 59], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 13, 'geese': [[54, 53, 52], [55, 66], [69], [15, 26]], 'food': [39, 59], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 14, 'geese': [[65, 54, 53], [44, 55], [58], [4,

{'remainingOverageTime': 60, 'step': 12, 'geese': [[23, 12], [], [13], [76]], 'food': [36, 18], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 13, 'geese': [[34, 23], [], [24], [10]], 'food': [36, 18], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 14, 'geese': [[33, 34], [], [35], [21]], 'food': [36, 18], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 15, 'geese': [[43, 33], [], [46], [32]], 'food': [36, 18], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 16, 'geese': [[54, 43], [], [45], [31]], 'food': [36, 18], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 17, 'geese': [[65, 54], [], [44], [30]], 'food': [36, 18], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 18, 'geese': [[76, 65], [], [54], [29]], 'food': [36, 18], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 19, 'geese': [[10, 76], [], [65], [40]], 'food': [36, 18], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 20,

Goose Collision: NORTH
{'remainingOverageTime': 60, 'step': 52, 'geese': [[47, 36, 37, 48, 59, 58, 57, 56], [], [], [76]], 'food': [27, 25], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 53, 'geese': [[46, 47, 36, 37, 48, 59, 58, 57], [], [], [75]], 'food': [27, 25], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 54, 'geese': [[45, 46, 47, 36, 37, 48, 59, 58], [], [], [64]], 'food': [27, 25], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 55, 'geese': [[56, 45, 46, 47, 36, 37, 48, 59], [], [], [53]], 'food': [27, 25], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 56, 'geese': [[55, 56, 45, 46, 47, 36, 37, 48], [], [], [54]], 'food': [27, 25], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 57, 'geese': [[66, 55, 56, 45, 46, 47, 36, 37], [], [], [65]], 'food': [27, 25], 'index': 0}
reward:  -1
{'remainingOverageTime': 60, 'step': 58, 'geese': [[0, 66, 55, 56, 45, 46, 47, 36], [], [], [76]], 'food': [27, 25], 'index':

### Ideas for future improvement:
